In [ ]:


import pandas as pd
import numpy as np
import os


import library as lib
from library import GlobalVars
from library import ExperimentConfig
import data_library as data_lib

import faiss_experiment as faiss_tool
import helpers.config_executor as executor


import experiments_configurations as ex_cfg
#
lib.init_random_seed(lib.RANDOM_SEED)

index_type="ivf_all"
faiss_experiments_output_file = GlobalVars.experiments_path + f"executed_faiss_experiments_index_{index_type}.csv"
all_faiss_experiments_file_path= GlobalVars.experiments_path + f"all_faiss_experiments_index_{index_type}.csv"
lib.combine_multiple_parameters_v3(all_faiss_experiments_file_path, ex_cfg.ivf_all)




In [ ]:
"""
Execute ONE configuration as test calling individual process
"""
os.environ["LOKY_MAX_CPU_COUNT"] = "8"
debug_mode=True
cfg_records=pd.read_csv(all_faiss_experiments_file_path)
row = cfg_records.iloc[0].copy()
new_row = executor.launch_execute_configurations_as_process(faiss_tool, row, debug_mode=debug_mode)
print("results=", new_row)

In [ ]:
"""
Execute ONE configuration as test by library function call
"""
debug_mode=True
cfg_records=pd.read_csv(all_faiss_experiments_file_path)
row = cfg_records.iloc[0].copy()
print(row)
print("index params=", row['index_params'], type(row["index_params"]))
cfg = ExperimentConfig(row)
GlobalVars.set_segment_lenght_and_overlap(cfg._SEGMENT_LENGHT, cfg._SEGMENT_OVERLAP)


faiss_results = faiss_tool.execute_configuration(cfg)
print("results=", faiss_results)

In [ ]:
'''
Execute all configurations from all_faiss_experiments_file_path
'''
do_normalize = True
debug_mode = False
repeat_no = 5
index_type = 'ivf_all'


all_faiss_experiments_file_path = GlobalVars.experiments_path + f"all_faiss_experiments_index_{index_type}.csv"


lib.combine_multiple_parameters_v3(all_faiss_experiments_file_path, ex_cfg.ivf_all)

for i in range(repeat_no):
    print(f"Execute round {i}")
    
    #you can cache the indexes before if you want
    #faiss_tool.cache_all_faiss_indexes(all_faiss_experiments_file_path)
        
    faiss_experiments_output_file = GlobalVars.experiments_path + f"executed_faiss_experiments_index_{index_type}_round{i}.csv"
    executor.process_all_configs_by_threads(exec_tool=faiss_tool, input_file_path=all_faiss_experiments_file_path, output_file_path=faiss_experiments_output_file, num_threads = 4, chunk_size = 20)
